In [4]:
# breakhistf2v28.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
  
%matplotlib inline

def read_csv(csvnamepath, filename):
     

    # read from csv file
    images, labels = [], []
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

 
def load_breakhis(dirc, mode='train'):
    classdir2label={}
    filename="tf2breakhisCSV"
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename )  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
@tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y
 

batchsz = 32
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')
db_train = tf.data.Dataset.from_tensor_slices((images, labels))  # 变成个Dataset对象。
db_train = db_train.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)  # map函数图片路径变为内容。
 

# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'val')
db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
db_val = db_val.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)
# create test db
images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'test')
db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
db_test = db_test.map(preprocess1).batch(batchsz)

 
resnetdense = keras.Sequential([
    layers.Conv2D(16,5,3),
    layers.MaxPool2D(3,3),
    layers.ReLU(),
    layers.Conv2D(64,5,3),
    layers.MaxPool2D(2,2),
    layers.ReLU(),
    
    layers.Flatten(),
    
    layers.Dense(128), 
    layers.Dropout(rate=0.5),    
    layers.Dense(64),  
    layers.Dropout(rate=0.5), 
    layers.Dense(32), 
    layers.Dropout(rate=0.5), 
    layers.Dense(16), 
    layers.Dropout(rate=0.5), 
    layers.Dense(8)
   ])


 

# 首先创建Resnet18
# resnet = ResNet(8)

resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

# monitor监听器, 连续5个验证准确率不增加，这个事情触发。
 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=100

)

  

 
# 网络的装配。
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

# 完成标准的train，val, test;
# 标准的逻辑必须通过db_val挑选模型的参数，就需要提供一个earlystopping技术，

LOGDIR='log/breakhistf2v28' 
resnetdense.fit(db_train, validation_data=db_val, 
                        validation_freq=10, epochs=2000, 
                        callbacks=[TensorBoard(log_dir=LOGDIR)])   # 1个epoch验证1次。触发了这个事情，提前停止了。
resnetdense.evaluate(db_test)
 
 

    
    
 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            multiple                  1216      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_9 (ReLU)               multiple                  0         
_________________________________________________________________
conv2d_10 (Conv2D)           multiple                  25664     
_________________________________________________________________
max_pooling2d_10 (MaxPooling multiple                  0         
_________________________________________________________________
re_lu_10 (ReLU)              multiple                  0         
_________________________________________________________________
flatten_3 (Flatten)          multiple                 

297/297 [==============================] - 191s 644ms/step - loss: 1.0941 - accuracy: 0.6158
Epoch 126/2000
297/297 [==============================] - 188s 632ms/step - loss: 1.0721 - accuracy: 0.6262
Epoch 127/2000
297/297 [==============================] - 187s 630ms/step - loss: 1.0800 - accuracy: 0.6240
Epoch 128/2000
297/297 [==============================] - 188s 635ms/step - loss: 1.0681 - accuracy: 0.6230
Epoch 129/2000
297/297 [==============================] - 197s 662ms/step - loss: 1.0595 - accuracy: 0.6289
Epoch 130/2000
297/297 [==============================] - 246s 829ms/step - loss: 1.0549 - accuracy: 0.6288 - val_loss: 1.0064 - val_accuracy: 0.6410
Epoch 131/2000
297/297 [==============================] - 195s 655ms/step - loss: 1.0557 - accuracy: 0.6301
Epoch 132/2000
297/297 [==============================] - 200s 672ms/step - loss: 1.0867 - accuracy: 0.6282
Epoch 133/2000
297/297 [==============================] - 191s 644ms/step - loss: 1.0493 - accuracy: 0.6321
E

Epoch 271/2000
297/297 [==============================] - 196s 660ms/step - loss: 0.8618 - accuracy: 0.6913
Epoch 272/2000
297/297 [==============================] - 198s 666ms/step - loss: 0.8502 - accuracy: 0.7013
Epoch 273/2000
297/297 [==============================] - 187s 629ms/step - loss: 0.8666 - accuracy: 0.6921
Epoch 274/2000
297/297 [==============================] - 190s 638ms/step - loss: 0.8448 - accuracy: 0.7006
Epoch 275/2000
297/297 [==============================] - 190s 640ms/step - loss: 0.8406 - accuracy: 0.6985
Epoch 276/2000
297/297 [==============================] - 187s 630ms/step - loss: 0.8561 - accuracy: 0.6991
Epoch 277/2000
297/297 [==============================] - 188s 632ms/step - loss: 0.8562 - accuracy: 0.6969
Epoch 278/2000
297/297 [==============================] - 188s 634ms/step - loss: 0.8553 - accuracy: 0.6942
Epoch 279/2000
297/297 [==============================] - 187s 630ms/step - loss: 0.8532 - accuracy: 0.6967
Epoch 280/2000
297/297 [====

297/297 [==============================] - 202s 680ms/step - loss: 0.7182 - accuracy: 0.7412
Epoch 418/2000
297/297 [==============================] - 202s 680ms/step - loss: 0.7243 - accuracy: 0.7370
Epoch 419/2000
297/297 [==============================] - 202s 679ms/step - loss: 0.7148 - accuracy: 0.7490
Epoch 420/2000
297/297 [==============================] - 263s 887ms/step - loss: 0.7118 - accuracy: 0.7433 - val_loss: 0.8536 - val_accuracy: 0.7241
Epoch 421/2000
297/297 [==============================] - 203s 684ms/step - loss: 0.6951 - accuracy: 0.7523
Epoch 422/2000
297/297 [==============================] - 201s 678ms/step - loss: 0.7068 - accuracy: 0.7519
Epoch 423/2000
297/297 [==============================] - 202s 681ms/step - loss: 0.6956 - accuracy: 0.7455
Epoch 424/2000
297/297 [==============================] - 205s 690ms/step - loss: 0.7145 - accuracy: 0.7498
Epoch 425/2000
297/297 [==============================] - 201s 676ms/step - loss: 0.6917 - accuracy: 0.7507
E

297/297 [==============================] - 204s 685ms/step - loss: 0.6177 - accuracy: 0.7771
Epoch 563/2000
297/297 [==============================] - 207s 696ms/step - loss: 0.6174 - accuracy: 0.7780
Epoch 564/2000
297/297 [==============================] - 205s 690ms/step - loss: 0.6330 - accuracy: 0.7747
Epoch 565/2000
297/297 [==============================] - 204s 689ms/step - loss: 0.6028 - accuracy: 0.7884
Epoch 566/2000
297/297 [==============================] - 203s 682ms/step - loss: 0.6066 - accuracy: 0.7803
Epoch 567/2000
297/297 [==============================] - 206s 692ms/step - loss: 0.6008 - accuracy: 0.7839
Epoch 568/2000
297/297 [==============================] - 204s 688ms/step - loss: 0.6098 - accuracy: 0.7817
Epoch 569/2000
297/297 [==============================] - 204s 688ms/step - loss: 0.6104 - accuracy: 0.7837
Epoch 570/2000
297/297 [==============================] - 266s 897ms/step - loss: 0.5976 - accuracy: 0.7865 - val_loss: 0.8165 - val_accuracy: 0.7377
E

...

297/297 [==============================] - 209s 704ms/step - loss: 0.3999 - accuracy: 0.8646
Epoch 1432/2000
297/297 [==============================] - 206s 694ms/step - loss: 0.3790 - accuracy: 0.8738
Epoch 1433/2000
297/297 [==============================] - 207s 697ms/step - loss: 0.3822 - accuracy: 0.8670
Epoch 1434/2000
297/297 [==============================] - 205s 691ms/step - loss: 0.4008 - accuracy: 0.8610
Epoch 1435/2000
297/297 [==============================] - 206s 695ms/step - loss: 0.4029 - accuracy: 0.8589
Epoch 1436/2000
297/297 [==============================] - 203s 684ms/step - loss: 0.3830 - accuracy: 0.8680
Epoch 1437/2000
297/297 [==============================] - 207s 698ms/step - loss: 0.3837 - accuracy: 0.8634
Epoch 1438/2000
297/297 [==============================] - 206s 692ms/step - loss: 0.4027 - accuracy: 0.8594
Epoch 1439/2000
297/297 [==============================] - 206s 694ms/step - loss: 0.3949 - accuracy: 0.8653
Epoch 1440/2000
297/297 [==========

297/297 [==============================] - 191s 643ms/step - loss: 0.3732 - accuracy: 0.8752
Epoch 1576/2000
297/297 [==============================] - 190s 639ms/step - loss: 0.3975 - accuracy: 0.8631
Epoch 1577/2000
297/297 [==============================] - 192s 646ms/step - loss: 0.3776 - accuracy: 0.8686
Epoch 1578/2000
297/297 [==============================] - 191s 643ms/step - loss: 0.3622 - accuracy: 0.8750
Epoch 1579/2000
297/297 [==============================] - 190s 641ms/step - loss: 0.3766 - accuracy: 0.8713
Epoch 1580/2000
297/297 [==============================] - 247s 831ms/step - loss: 0.3829 - accuracy: 0.8698 - val_loss: 1.0641 - val_accuracy: 0.7636
Epoch 1581/2000
297/297 [==============================] - 191s 644ms/step - loss: 0.3801 - accuracy: 0.8692
Epoch 1582/2000
297/297 [==============================] - 189s 636ms/step - loss: 0.3738 - accuracy: 0.8682
Epoch 1583/2000
297/297 [==============================] - 190s 640ms/step - loss: 0.3783 - accuracy: 

297/297 [==============================] - 207s 698ms/step - loss: 0.3546 - accuracy: 0.8689
Epoch 1720/2000
297/297 [==============================] - 267s 898ms/step - loss: 0.3551 - accuracy: 0.8765 - val_loss: 0.9832 - val_accuracy: 0.7822
Epoch 1721/2000
297/297 [==============================] - 209s 702ms/step - loss: 0.3686 - accuracy: 0.8760
Epoch 1722/2000
297/297 [==============================] - 202s 682ms/step - loss: 0.3662 - accuracy: 0.8720
Epoch 1723/2000
297/297 [==============================] - 204s 688ms/step - loss: 0.3671 - accuracy: 0.8724
Epoch 1724/2000
297/297 [==============================] - 204s 688ms/step - loss: 0.3621 - accuracy: 0.8757
Epoch 1725/2000
297/297 [==============================] - 203s 683ms/step - loss: 0.3616 - accuracy: 0.8747
Epoch 1726/2000
297/297 [==============================] - 207s 697ms/step - loss: 0.3680 - accuracy: 0.8733
Epoch 1727/2000
297/297 [==============================] - 203s 682ms/step - loss: 0.3650 - accuracy: 

297/297 [==============================] - 188s 633ms/step - loss: 0.3346 - accuracy: 0.8839
Epoch 1864/2000
297/297 [==============================] - 189s 635ms/step - loss: 0.3444 - accuracy: 0.8805
Epoch 1865/2000
297/297 [==============================] - 189s 636ms/step - loss: 0.3656 - accuracy: 0.8739
Epoch 1866/2000
297/297 [==============================] - 188s 633ms/step - loss: 0.3628 - accuracy: 0.8782
Epoch 1867/2000
297/297 [==============================] - 189s 637ms/step - loss: 0.3542 - accuracy: 0.8787
Epoch 1868/2000
297/297 [==============================] - 188s 633ms/step - loss: 0.3420 - accuracy: 0.8834
Epoch 1869/2000
297/297 [==============================] - 188s 634ms/step - loss: 0.3525 - accuracy: 0.8785
Epoch 1870/2000
297/297 [==============================] - 250s 843ms/step - loss: 0.3558 - accuracy: 0.8777 - val_loss: 0.9643 - val_accuracy: 0.7750
Epoch 1871/2000
297/297 [==============================] - 188s 635ms/step - loss: 0.3487 - accuracy: 

[1.0757045561075211, 0.76675093]

In [4]:
import tensorflow as tf
model.save_weights('./checkpoints/mannul_checkpoint')
 

NameError: name 'model' is not defined